In [1]:
from pathlib import Path
import pandas as pd

def load_abcd_table(path):
    table = pd.read_csv(path, skiprows=[1], header=0, sep='\t')
    labels = pd.read_csv(path, nrows=1, header=0, sep='\t')
    labels = labels.T.reset_index().rename(columns={'index':'name', 0:'doc'})
    return table, labels


In [2]:
release_dir = Path('/abcdqc_data/releases/1.1/ABCDstudyDEAP')

In [3]:
# get the list of subjects in the most recent release
# We're assuming that all subjects are in the site table
site_df = pd.read_csv(release_dir / 'abcd_lt01.txt',
                      skiprows = [1],
                      header = 0,
                      sep='\t')

In [4]:
site_df['bids_meta__subject_id'] = site_df.subjectkey.str.replace('_','')

In [5]:
json_dat = pd.read_csv('/abcdqc_data/batchserver/output/df.csv')
json_dat.columns = json_dat.columns.str.replace('.', '__')

In [6]:
to_merge = site_df.loc[:,['interview_age', 'gender', 'bids_meta__subject_id']]
mriqc_merge = json_dat.merge(to_merge, how='left', on='bids_meta__subject_id', indicator=True)
assert mriqc_merge.groupby('_merge').provenance__md5sum.count()['right_only'] == 0
assert mriqc_merge.groupby('_merge').provenance__md5sum.count()['both'] != 0
assert mriqc_merge.groupby('_merge').provenance__md5sum.count()['left_only'] != 0

In [7]:
# Get the FreesurferQC
fsqc, fsqclbl = load_abcd_table(release_dir / 'freesqc01.txt')
mriqc, mriqclbl = load_abcd_table(release_dir / 'mriqc02.txt')
mriqc2, mriqclbl = load_abcd_table(release_dir / 'mriqcp202.txt')
midbeh, _ = load_abcd_table(release_dir / 'abcd_mid02.txt')
nbbeh, _ = load_abcd_table(release_dir / 'abcd_mrinback02.txt')
sstbeh, _ = load_abcd_table(release_dir / 'abcd_sst02.txt')
mrfind, _ = load_abcd_table(release_dir / 'abcd_mrfindings01.txt')


/home/nielsond/miniconda3/envs/abcdqc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (96,97,496,497,523,524,550,551,854,855,881,882) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/home/nielsond/miniconda3/envs/abcdqc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (116,117,143,144,339,340,366,367,393,394,736,742,760,776,778,788,790,798,800,802) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [8]:
image_tbls = ["abcd_smrip101", "abcd_smrip201", "abcd_dti_p101", "abcd_dti_p201",
 "mri_rsi_p102", "mri_rsi_p202", "abcd_midr1bwp101", "abcd_midr1bwp201",
 "midr2bwp101", "midr2bwp201", "mrisstr1bw01", "mrisstr2bw01", "nbackr101",
 "nbackr201", "abcd_mrirstv02", "midaparc02", "midaparcp202", 
 "abcd_midasemp101", "abcd_midasemp201", "abcd_midsemp101",
 "abcd_midsemp201", "abcd_midr2semp101", "abcd_midr2semp201",
 "mrisst02", "mrisstsem01", "mrisstr1sem01", "mrisstr2bwsem01",
 "nback_bwroi02", "nbackallsem01", "nbackr1sem01", "nbackr2sem01",
 "dmriqc01", "abcd_mid02", "abcd_sst02", "abcd_mrinback02", "mribrec02",
 "abcd_mrfindings01", "mriqc02", "mriqcp202", "freesqc01"]
non_deskian_tables = ["abcd_mrirstv02", "dmriqc01", "abcd_mid02", "abcd_sst02", "abcd_mrinback02", "mribrec02",
 "abcd_mrfindings01", "mriqc02", "mriqcp202", "freesqc01"]
deskiab_tbls = [it for it in image_tbls if it not in non_deskian_tables]
imgtbls = {}
imglbls = []
for itn in deskiab_tbls:
    imgtbl, imglbl = load_abcd_table(release_dir / (itn + '.txt'))
    imglbl['source_file'] = itn
    imgtbls[itn] = imgtbl
    imglbls.append(imglbl)

In [9]:
longtbl = fsqc

longtbl = longtbl.merge(mriqc.loc[:, ['subjectkey', 'interview_date', 'eventname', 'iqc_t2_ok_ser', 'iqc_mid_ok_ser']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

longtbl = longtbl.merge(mriqc2.loc[:, ['subjectkey', 'interview_date', 'iqc_sst_ok_ser', 'iqc_nback_ok_ser']],
                how='left',
                on=['subjectkey','interview_date'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

longtbl = longtbl.merge(midbeh.loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_mid_beh_perform.flag']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(nbbeh.loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_nback_beh_perform.flag']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(sstbeh.loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_sst_beh_perform.flag']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

longtbl = longtbl.merge(imgtbls['midaparc02'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_mid_all_beta_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(imgtbls['abcd_midasemp101'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_mid_all_sem_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)


longtbl = longtbl.merge(imgtbls['mrisst02'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_sst_all_beta_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(imgtbls['mrisstsem01'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_sst_all_sem_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)


longtbl = longtbl.merge(imgtbls['nback_bwroi02'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_nback_all_beta_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)
longtbl = longtbl.merge(imgtbls['nbackallsem01'].loc[:, ['subjectkey', 'interview_date', 'eventname', 'tfmri_nback_all_sem_dof']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

longtbl = longtbl.merge(mrfind.loc[:, ['subjectkey', 'interview_date', 'eventname', 'mrif_score', 'mrif_hydrocephalus', 'mrif_herniation']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

bn_t, bn_l = load_abcd_table(release_dir / 'abcd_betnet02.txt')
# Network to subcortical table
ns_t, ns_l = load_abcd_table(release_dir /  'mrirscor02.txt')
# We'll merge on subjectkey and interview_date, filter out the other columns from ns_t before merging
ns_t.drop(ns_l.loc[ns_l.doc.isin(bn_l.doc),'name'].drop([3,5]).values,
          axis=1,
          inplace=True)

con = bn_t.merge(ns_t,
                 how='outer',
                 on=['subjectkey','interview_date'])

con_l = pd.concat([bn_l, 
                   ns_l[~ns_l.name.isin(bn_l.loc[bn_l.doc.isin(ns_l.doc),'name'].values)]]).reset_index(drop=True)
longtbl = longtbl.merge(con.loc[:, ['subjectkey', 'interview_date', 'eventname','rsfmri_cor_network.gordon_ntpoints']],
                how='left',
                on=['subjectkey','interview_date', 'eventname'],
                indicator=True)
assert longtbl.groupby('_merge')['collection_id'].count()['both'] == len(longtbl)
longtbl.drop('_merge', axis=1, inplace=True)

In [10]:
longtbl['mr_findings_ok'] = (longtbl['mrif_score'] < 3)  & (longtbl['mrif_hydrocephalus'] == 'no') &  (longtbl['mrif_herniation'] == 'no')


In [11]:
# Default everyting to qc_ok as False
longtbl['qc_ok'] = False

In [12]:
longtbl['bids_meta__subject_id'] = longtbl.subjectkey.str.replace('_','')

In [13]:
mriqc_long = mriqc_merge.merge(longtbl, how='left', on='bids_meta__subject_id', indicator='long_merge')
assert mriqc_long.shape[0] == mriqc_long.shape[0]

In [14]:
mriqc_long.groupby('long_merge').provenance__md5sum.count()

long_merge
left_only     12726
right_only        0
both          33760
Name: provenance__md5sum, dtype: int64

In [15]:
mriqc_long.bids_meta__modality.unique()

array(['T1w', 'T2w', 'bold'], dtype=object)

In [16]:
# QC qualifications for smri

t1_metrics = ['area', 'sulc', 't1w', 't1w.contrast', 't1w.gray02', 't1w.white02', 'thick', 'vol']
t2_metrics = ['t2w', 't2w.contrast', 't2w.gray02', 't2w.white02']


mriqc_long.loc[((mriqc_long.bids_meta__modality == 'T1w') 
              & (mriqc_long.fsqc_qc == 1)), 'qc_ok'] = True


mriqc_long.loc[((mriqc_long.bids_meta__modality == 'T2w') 
              & (mriqc_long.fsqc_qc == 1)), 'qc_ok'] = True

In [17]:
# QC for MID task
mriqc_long.loc[((mriqc_long.bids_meta__modality == 'bold') 
             & (mriqc_long.bids_meta__TaskName == 'mid')
             & (mriqc_long.fsqc_qc == 1)
             & (mriqc_long.iqc_mid_ok_ser > 0)
             & (mriqc_long['tfmri_mid_beh_perform.flag'] == 1)
             & (mriqc_long.tfmri_mid_all_beta_dof > 200)
             & (mriqc_long.tfmri_mid_all_sem_dof > 200)), 'qc_ok'] = True

# QC for SST
mriqc_long.loc[((mriqc_long.bids_meta__modality == 'bold') 
             & (mriqc_long.bids_meta__TaskName == 'sst')
             & (mriqc_long.fsqc_qc == 1)
             & (mriqc_long.iqc_sst_ok_ser > 0)
             & (mriqc_long['tfmri_sst_beh_perform.flag'] == 1)
             & (mriqc_long.tfmri_sst_all_beta_dof > 200)
             & (mriqc_long.tfmri_sst_all_sem_dof > 200)), 'qc_ok'] = True

# QC for nBack
mriqc_long.loc[((mriqc_long.bids_meta__modality == 'bold') 
             & (mriqc_long.bids_meta__TaskName == 'nback')
             & (mriqc_long.fsqc_qc == 1)
             & (mriqc_long.iqc_nback_ok_ser > 0)
             & (mriqc_long['tfmri_nback_beh_perform.flag'] == 1)
             & (mriqc_long.tfmri_nback_all_beta_dof > 200)
             & (mriqc_long.tfmri_nback_all_sem_dof > 200)), 'qc_ok'] = True

# QC for rest
mriqc_long.loc[((mriqc_long.bids_meta__modality == 'bold') 
             & (mriqc_long.bids_meta__TaskName == 'rest')
             & (mriqc_long['rsfmri_cor_network.gordon_ntpoints'] > 375)), 'qc_ok'] = True

In [20]:
mriqc_all = mriqc_merge.merge(mriqc_long.loc[:,['provenance__md5sum', 'qc_ok']], on='provenance__md5sum', indicator='all_merge')
assert mriqc_all.groupby('all_merge').provenance__md5sum.count()['right_only'] == 0
assert mriqc_all.groupby('all_merge').provenance__md5sum.count()['both'] != 0
assert mriqc_all.groupby('all_merge').provenance__md5sum.count()['left_only'] == 0

In [21]:
mriqc_all.drop(['_merge', 'all_merge'], axis=1)

,aor,aqi,bids_meta__AcquisitionMatrixPE,bids_meta__AcquisitionNumber,bids_meta__AcquisitionTime,bids_meta__BandwidthPerPixelPhaseEncode,bids_meta__BaseResolution,bids_meta__BodyPartExamined,bids_meta__ConsistencyInfo,bids_meta__ConversionSoftware,...,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,tsnr,wm2max,interview_age,gender,qc_ok
0,NaN,NaN,256,1,15:23:23.090000,NaN,256.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,1204.535242,97.821877,0.216683,0.520474,0.506862,NaN,0.593835,119.0,F,True
1,NaN,NaN,256,1,15:54:40.162500,NaN,256.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,3029.850049,814.579285,0.119401,0.282831,0.200539,NaN,0.320484,119.0,F,True
2,0.000739,0.006520,90,1,15:32:30.265000,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,37.171486,NaN,119.0,F,True
3,0.002386,0.006353,90,1,16:23:23.760000,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,36.050770,NaN,119.0,F,True
4,0.002055,0.008803,90,1,16:37:9.682500,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,31.893375,NaN,119.0,F,True
5,0.003376,0.007049,90,1,15:38:25.302500,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,37.347023,NaN,119.0,F,True
6,0.001880,0.008985,90,1,16:03:58.392500,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,33.200676,NaN,119.0,F,True
7,0.002237,0.006321,90,1,16:45:28.025000,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,37.130959,NaN,119.0,F,True
8,0.001150,0.007143,90,1,16:30:34.107500,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,35.399441,NaN,119.0,F,True
9,0.005954,0.010058,90,1,16:09:40.480000,21.786,90.0,BRAIN,N4_VE11C_LATEST_20160120,dcm2niix,...,NaN,NaN,NaN,NaN,NaN,31.954880,NaN,119.0,F,True


In [23]:
mriqc_all.to_csv('/abcdqc_data/batchserver/output/df_plus_meta.csv', index=False)

In [24]:
iqms = [ 'cjv', 'cnr', 'efc', 'fber', 'fwhm_avg', 'fwhm_x', 'fwhm_y', 'fwhm_z', 'icvs_csf', 'icvs_gm', 'icvs_wm', 'inu_med', 'inu_range',
      'qi_1', 'qi_2', 'rpve_csf', 'rpve_gm', 'rpve_wm', 'size_x', 'size_y', 'size_z', 'snr_csf', 'snr_gm',
      'snr_total', 'snr_wm', 'snrd_csf', 'snrd_gm', 'snrd_total', 'snrd_wm', 'spacing_x', 'spacing_y', 'spacing_z', 'summary_bg_k',
      'summary_bg_mad', 'summary_bg_mean', 'summary_bg_median', 'summary_bg_n', 'summary_bg_p05', 'summary_bg_p95',
      'summary_bg_stdv', 'summary_csf_k', 'summary_csf_mad', 'summary_csf_mean', 'summary_csf_median', 'summary_csf_n',
      'summary_csf_p05', 'summary_csf_p95', 'summary_csf_stdv', 'summary_gm_k', 'summary_gm_mad', 'summary_gm_mean',
      'summary_gm_median', 'summary_gm_n', 'summary_gm_p05', 'summary_gm_p95', 'summary_gm_stdv', 'summary_wm_k',
      'summary_wm_mad', 'summary_wm_mean', 'summary_wm_median', 'summary_wm_n', 'summary_wm_p05', 'summary_wm_p95',
      'summary_wm_stdv', 'tpm_overlap_csf', 'tpm_overlap_gm', 'tpm_overlap_wm', 'wm2max' ]

In [44]:
t1_iqms = (mriqc_all.query('bids_meta__modality == "T1w"').describe().sum(0) != 0)
#t1_iqms.index[t1_iqms]
t2_iqms = (mriqc_all.query('bids_meta__modality == "T2w"').describe().sum(0) != 0)
#t2_iqms.index[t2_iqms]
bold_iqms = (mriqc_all.query('bids_meta__modality == "bold"').describe().sum(0) != 0)
bold_iqms.index[bold_iqms]

Index(['aor', 'aqi', 'bids_meta__AcquisitionMatrixPE',
       'bids_meta__AcquisitionNumber',
       'bids_meta__BandwidthPerPixelPhaseEncode', 'bids_meta__BaseResolution',
       'bids_meta__DerivedVendorReportedEchoSpacing', 'bids_meta__DwellTime',
       'bids_meta__EchoTime', 'bids_meta__EchoTrainLength',
       'bids_meta__EffectiveEchoSpacing', 'bids_meta__FlipAngle',
       'bids_meta__MagneticFieldStrength',
       'bids_meta__MultibandAccelerationFactor', 'bids_meta__PartialFourier',
       'bids_meta__PercentPhaseFOV', 'bids_meta__PhaseEncodingSteps',
       'bids_meta__PhaseResolution', 'bids_meta__PhilipsRescaleIntercept',
       'bids_meta__PhilipsRescaleSlope', 'bids_meta__PhilipsScaleSlope',
       'bids_meta__PixelBandwidth', 'bids_meta__ReconMatrixPE',
       'bids_meta__RepetitionTime', 'bids_meta__SAR',
       'bids_meta__SeriesNumber', 'bids_meta__SliceThickness',
       'bids_meta__SpacingBetweenSlices', 'bids_meta__TotalReadoutTime',
       'bids_meta__TxRefAmp', 

In [ ]:
t1wiqms = ['cjv', 'cnr', 'efc', 'fber', 'fwhm_avg',
       'fwhm_x', 'fwhm_y', 'fwhm_z', 'icvs_csf', 'icvs_gm', 'icvs_wm',
       'inu_med', 'inu_range', 'qi_1', 'qi_2', 'rpve_csf', 'rpve_gm',
       'rpve_wm', 'size_x', 'size_y', 'size_z', 'snr_csf', 'snr_gm',
       'snr_total', 'snr_wm', 'snrd_csf', 'snrd_gm', 'snrd_total', 'snrd_wm',
       'spacing_x', 'spacing_y', 'spacing_z', 'summary_bg_k', 'summary_bg_mad',
       'summary_bg_mean', 'summary_bg_median', 'summary_bg_n',
       'summary_bg_p05', 'summary_bg_p95', 'summary_bg_stdv', 'summary_csf_k',
       'summary_csf_mad', 'summary_csf_mean', 'summary_csf_median',
       'summary_csf_n', 'summary_csf_p05', 'summary_csf_p95',
       'summary_csf_stdv', 'summary_gm_k', 'summary_gm_mad', 'summary_gm_mean',
       'summary_gm_median', 'summary_gm_n', 'summary_gm_p05', 'summary_gm_p95',
       'summary_gm_stdv', 'summary_wm_k', 'summary_wm_mad', 'summary_wm_mean',
       'summary_wm_median', 'summary_wm_n', 'summary_wm_p05', 'summary_wm_p95',
       'summary_wm_stdv', 'tpm_overlap_csf', 'tpm_overlap_gm',
       'tpm_overlap_wm', 'wm2max']

t2wiqms = ['cjv', 'cnr', 'efc', 'fber', 'fwhm_avg',
       'fwhm_x', 'fwhm_y', 'fwhm_z', 'icvs_csf', 'icvs_gm', 'icvs_wm',
       'inu_med', 'inu_range', 'qi_1', 'qi_2', 'rpve_csf', 'rpve_gm',
       'rpve_wm', 'size_x', 'size_y', 'size_z', 'snr_csf', 'snr_gm',
       'snr_total', 'snr_wm', 'snrd_csf', 'snrd_gm', 'snrd_total', 'snrd_wm',
       'spacing_x', 'spacing_y', 'spacing_z', 'summary_bg_k', 'summary_bg_mad',
       'summary_bg_mean', 'summary_bg_median', 'summary_bg_n',
       'summary_bg_p05', 'summary_bg_p95', 'summary_bg_stdv', 'summary_csf_k',
       'summary_csf_mad', 'summary_csf_mean', 'summary_csf_median',
       'summary_csf_n', 'summary_csf_p05', 'summary_csf_p95',
       'summary_csf_stdv', 'summary_gm_k', 'summary_gm_mad', 'summary_gm_mean',
       'summary_gm_median', 'summary_gm_n', 'summary_gm_p05', 'summary_gm_p95',
       'summary_gm_stdv', 'summary_wm_k', 'summary_wm_mad', 'summary_wm_mean',
       'summary_wm_median', 'summary_wm_n', 'summary_wm_p05', 'summary_wm_p95',
       'summary_wm_stdv', 'tpm_overlap_csf', 'tpm_overlap_gm',
       'tpm_overlap_wm', 'wm2max']

boldiqms = ['dummy_trs', 'dvars_nstd',
       'dvars_std', 'dvars_vstd', 'efc', 'fber', 'fd_mean', 'fd_num',
       'fd_perc', 'fwhm_avg', 'fwhm_x', 'fwhm_y', 'fwhm_z', 'gcor', 'gsr_x',
       'gsr_y', 'provenance__settings__fd_thres', 'size_t', 'size_x', 'size_y',
       'size_z', 'snr', 'spacing_tr', 'spacing_x', 'spacing_y', 'spacing_z',
       'summary_bg_k', 'summary_bg_mad', 'summary_bg_mean',
       'summary_bg_median', 'summary_bg_n', 'summary_bg_p05', 'summary_bg_p95',
       'summary_bg_stdv', 'summary_fg_k', 'summary_fg_mad', 'summary_fg_mean',
       'summary_fg_median', 'summary_fg_n', 'summary_fg_p05', 'summary_fg_p95',
       'summary_fg_stdv', 'tsnr']